In [ ]:
rows1 = int(input("Enter the number of rows for matrix 1: "))
cols1 = int(input("Enter the number of columns for matrix 1: "))
inputs = []
for i in range(rows1):
    row = []
    for j in range(cols1):
        element = int(input("Enter element at position ({}, {}): ".format(i, j)))
        row.append(element)
    inputs.append(row)

for row in inputs:
    print(row)


Enter the number of rows for matrix 1: 4
Enter the number of columns for matrix 1: 1
Enter element at position (0, 0): 1
Enter element at position (1, 0): 2
Enter element at position (2, 0): 3
Enter element at position (3, 0): 4
[1]
[2]
[3]
[4]


In [ ]:
import numpy as np

# Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# Initialize weights and bias
def initialize_weights(input_size):
    return np.random.uniform(-1, 1, (input_size, 1))

def initialize_bias():
    return np.random.uniform(-1, 1)

# Forward pass with bias
def forward(inputs, weights, bias):
    return sigmoid(np.dot(inputs, weights) + bias)

# Training function with bias adjustment
def train(training_inputs, training_outputs, training_iterations, weights, bias):

    for iteration in range(training_iterations):
        output = forward(training_inputs, weights, bias)
        error = training_outputs - output

        adjustments = np.dot(training_inputs.T, error * sigmoid_derivative(output))
        weights += adjustments

    return weights, bias

# Training data
training_inputs = np.array([[0, 0, 1],
                            [1, 1, 1],
                            [1, 0, 1],
                            [0, 1, 1]])

training_outputs = np.array([[0, 1, 1, 0]]).T

# Initialize weights and bias
weights = initialize_weights(3)
bias = initialize_bias()

print("Initial weights:")
print(weights)
print("Initial bias:")
print(bias)

# Train the neural network
trained_weights, trained_bias = train(training_inputs, training_outputs, 10000, weights, bias)

print("Trained weights after training:")
print(trained_weights)

new_input = np.array([1, 0, 0])
output = forward(new_input, trained_weights, trained_bias)
print("Prediction for new input [1, 0, 0]:")
print(output)


Initial weights:
[[ 0.95250972]
 [-0.33258866]
 [ 0.79324318]]
Initial bias:
-0.9504991451342271
Trained weights after training:
[[ 9.67332988]
 [-0.20858297]
 [-3.67886503]]
Prediction for new input [1, 0, 0]:
[0.9998372]


In [ ]:
def encoder_block(inputs, num_filters):

	# Convolution with 3x3 filter followed by ReLU activation
	x = tf.keras.layers.Conv2D(num_filters,
							3,
							padding = 'valid')(inputs)
	x = tf.keras.layers.Activation('relu')(x)

	# Convolution with 3x3 filter followed by ReLU activation
	x = tf.keras.layers.Conv2D(num_filters,
							3,
							padding = 'valid')(x)
	x = tf.keras.layers.Activation('relu')(x)

	# Max Pooling with 2x2 filter
	x = tf.keras.layers.MaxPool2D(pool_size = (2, 2),
								strides = 2)(x)

	return x
def decoder_block(inputs, skip_features, num_filters):

	# Upsampling with 2x2 filter
	x = tf.keras.layers.Conv2DTranspose(num_filters,
										(2, 2),
										strides = 2,
										padding = 'valid')(inputs)

	# Copy and crop the skip features
	# to match the shape of the upsampled input
	skip_features = tf.image.resize(skip_features,
									size = (x.shape[1],
											x.shape[2]))
	x = tf.keras.layers.Concatenate()([x, skip_features])

	# Convolution with 3x3 filter followed by ReLU activation
	x = tf.keras.layers.Conv2D(num_filters,
							3,
							padding = 'valid')(x)
	x = tf.keras.layers.Activation('relu')(x)

	# Convolution with 3x3 filter followed by ReLU activation
	x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'valid')(x)
	x = tf.keras.layers.Activation('relu')(x)

	return x
# Unet code
import tensorflow as tf

def unet_model(input_shape = (256, 256, 3), num_classes = 1):
	inputs = tf.keras.layers.Input(input_shape)

	# Contracting Path
	s1 = encoder_block(inputs, 64)
	s2 = encoder_block(s1, 128)
	s3 = encoder_block(s2, 256)
	s4 = encoder_block(s3, 512)

	# Bottleneck
	b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(s4)
	b1 = tf.keras.layers.Activation('relu')(b1)
	b1 = tf.keras.layers.Conv2D(1024, 3, padding = 'valid')(b1)
	b1 = tf.keras.layers.Activation('relu')(b1)

	# Expansive Path
	s5 = decoder_block(b1, s4, 512)
	s6 = decoder_block(s5, s3, 256)
	s7 = decoder_block(s6, s2, 128)
	s8 = decoder_block(s7, s1, 64)

	# Output
	outputs = tf.keras.layers.Conv2D(num_classes,
									1,
									padding = 'valid',
									activation = 'sigmoid')(s8)

	model = tf.keras.models.Model(inputs = inputs,
								outputs = outputs,
								name = 'U-Net')
	return model

if __name__ == '__main__':
	model = unet_model(input_shape=(256, 256, 3), num_classes=2)
	model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
